In [72]:
import numpy as np 
import pandas as pd

In [73]:
url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dados_multi_classe.csv"
df = pd.read_csv(url, delimiter=',')

In [74]:
df

,Institution,Affiliations
0,aristotle university greece,1st department of ophthalmology aristotle univ...
1,aristotle university greece,aristotle univ thessaloniki greece
2,aristotle university greece,aristotle univ thessaloniki thessaloniki greece
3,aristotle university greece,aristotle university of thessaloniki greece
4,aristotle university greece,aristotle university of thessaloniki thessalon...
...,...,...
122929,universidade federal reconcavo,"univ fed reconcavo bahia, centro formacao prof..."
122930,unesp,unesp-marilia
122931,universidade federal campina grande,univ federal campina grande
122932,revista saude publica,revista de saude publica


In [75]:
fiocruz = df[df['Institution'].str.contains("fiocruz")]

In [76]:
fiocruz.shape[0]

20745

In [77]:
fiocruz.sample(n=10)

,Institution,Affiliations
15254,ensp escola nacional de saúde pública sérgio ...,oswaldo cruz foundation national school of pub...
42199,incqs instituto nacional de controle de quali...,national institute of quality control in healt...
14505,ensp escola nacional de saúde pública sérgio ...,fundacao oswaldo cruz fiocruz escola nacional...
12979,ensp escola nacional de saúde pública sérgio ...,departamento de endemias samuel pessoa escola ...
28060,fiocruz ro,fundação oswaldo cruzrondônia porto velho ro b...
13669,ensp escola nacional de saúde pública sérgio ...,escola nacional de saude publica sergio arouca...
44363,ini instituto nacional de infectologia evandr...,laboratorio de micologia setor de imunodiagnos...
13575,ensp escola nacional de saúde pública sérgio ...,ensp fiocruz imsuerj brazil
19272,fiocruz am instituto leônidas e maria deane ...,postgraduate program in living conditions and ...
44795,ini instituto nacional de infectologia evandr...,laboratório de pesquisa clínica em doença de c...


In [78]:
df = fiocruz.to_csv("dicionario_fiocruz.tsv", sep = "\t", index=False)

In [79]:
url = "/content/dicionario_fiocruz.tsv"
df = pd.read_csv(url, delimiter='\t')

In [80]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image 
from six import StringIO
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [81]:
url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dicionario_fiocruz.tsv"
df = pd.read_csv(url, delimiter='\t')

In [82]:
df.sample(n=3)

,Institution,Affiliations
5986,fiocruz am instituto leônidas e maria deane ...,instituto leônidas e maria deane fundação oswa...
17403,incqs instituto nacional de controle de quali...,departamento de química instituto nacional de ...
3923,ensp escola nacional de saúde pública sérgio ...,national school of public health sergio arouca...


In [83]:
X, y = df['Affiliations'],df['Institution']
X.shape, y.shape

((20745,), (20745,))

In [84]:
X

0                                      biblioteca  fiocruz
1        biblioteca de manguinhos avenida brasil 4365 m...
2        department of health information oswaldo cruz ...
3                        fiocruz  biblioteca de manguinhos
4        health information fundação oswaldo cruz bibli...
                               ...                        
20740    scientific computing program oswaldo cruz foun...
20741    scientific computing programoswaldo cruz found...
20742    scientific computing programoswaldo cruz found...
20743    scientific computing programme fundação oswald...
20744    serviço de jornalismo e comunicação programa d...
Name: Affiliations, Length: 20745, dtype: object

In [85]:
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
palavrasIgnoradas = \
    stopwords.words('portuguese') + \
    stopwords.words('english') + \
    stopwords.words('german') + \
    stopwords.words('spanish')

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(
    max_features = 1500, # vocabulário considera top max_features ordenadas por freq do termo no texto 
    min_df = 5, # ignorar termos com frequencia menr do que x, onde 0<=x<=1
    max_df = 0.7, # ignorar termos com frequencia maior do que x, onde 0<=x<=1
    stop_words = palavrasIgnoradas
)

In [88]:
vectorizer.fit_transform(X)

<20745x1483 sparse matrix of type '<class 'numpy.int64'>'
	with 232127 stored elements in Compressed Sparse Row format>

In [89]:
X = vectorizer.fit_transform(X).toarray()

In [90]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [91]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=0, shuffle=True)

In [92]:
print(X_treino.shape)
print(X_teste.shape)

(18670, 1483)
(2075, 1483)


#K-Nearest Neighbors

In [93]:
knn = KNeighborsClassifier()
knn.fit(X_treino, y_treino)
resultado_knn = knn.predict(X_teste)

In [94]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_knn) * 100}")

Acurácia: 96.67469879518073


# DecisionTree

In [95]:
arvore = DecisionTreeClassifier(criterion='entropy' ,random_state=0)
dt = arvore.fit(X_treino, y_treino)
resultado_dt = dt.predict(X_teste)

In [96]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_dt) * 100}")

Acurácia: 94.74698795180723


# RandomForest

In [97]:
floresta = RandomForestClassifier(criterion='entropy', random_state=0)
floresta.fit(X_treino, y_treino)
resultado_rd = floresta.predict(X_teste)

In [98]:
print(f"Acurácia: {metrics.accuracy_score(y_teste, resultado_rd) * 100}")

Acurácia: 97.44578313253012


# MultinomialNB

In [99]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_treino, y_treino)
resultado_nb = nb.predict(X_teste)

In [100]:
print(f"Acurácia: {accuracy_score(y_teste, resultado_nb) * 100}")

Acurácia: 95.46987951807229


#SVM Kernel Linear

In [101]:
svm_kernel_linear = svm.SVC(kernel = 'linear')
prob_linear = svm_kernel_linear.fit(X_treino, y_treino)

In [102]:
pred_svm_linear = svm_kernel_linear.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_linear) * 100)

Acurácia = 98.02409638554217


#SVM Kernel RBF

In [103]:
svm_kernel_rbf = svm.SVC(kernel = 'rbf')
prob_rbf = svm_kernel_rbf.fit(X_treino, y_treino)

In [104]:
pred_svm_rbf = svm_kernel_rbf.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_rbf) * 100)

Acurácia = 97.87951807228914


#SVM Kernel Sigmoid

In [105]:
svm_kernel_sigmoid = svm.SVC(kernel = 'sigmoid')
prob_sigmoid = svm_kernel_sigmoid.fit(X_treino, y_treino)

In [106]:
pred_svm_sigmoid = svm_kernel_sigmoid.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_sigmoid) * 100)

Acurácia = 97.87951807228914


#SVM Kernel Poly

In [107]:
svm_kernel_poly = svm.SVC(kernel = 'poly')
prob_poly = svm_kernel_poly.fit(X_treino, y_treino)

In [108]:
pred_svm_poly = svm_kernel_poly.predict(X_teste)
print("Acurácia =", accuracy_score(y_teste, pred_svm_poly) * 100)

Acurácia = 97.01204819277109


#Teste

In [109]:
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn. feature_extraction.text import TfidfVectorizer
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn import svm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
palavrasIgnoradas = \
    stopwords.words('portuguese') + \
    stopwords.words('english') + \
    stopwords.words('german') + \
    stopwords.words('spanish')

In [111]:
from sklearn. feature_extraction.text import TfidfVectorizer
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn import svm

url = "https://raw.githubusercontent.com/joaograndotto/PIBITI/main/Datasets/dicionario_fiocruz.tsv"
df = pd.read_csv(url, delimiter='\t')


model = Pipeline ([('tfidf', TfidfVectorizer(max_features = 1500, 
    min_df = 5, 
    max_df = 0.7,
    stop_words = palavrasIgnoradas)), ('logreg', svm.SVC(kernel = 'linear'))])
model.fit(df['Affiliations'], df['Institution'])
Pipeline(steps=[('tfidf', TfidfVectorizer()), ('logreg', svm.SVC(kernel = 'linear'))])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('logreg', SVC(kernel='linear'))])

In [112]:
test_dict = {'Goncalo Moniz': 'Instituto Goncalo Moniz Fiocruz Bahia',
'Ageu Magalhaes':'Instituto Aggeu Magalhães',
'Fnd Osw. Cruz':'Fundacao Oswaldo Cruz',
'Fundacao Osvaldo Crus':'Fundacao Oswaldo Cruz',
'Fnd Osw. Cruz':'Fiocruz',
'Instituto Leônidas e Maria Deane':'Fiocruz Amazônia',
'Instituto René Rachou':'Fiocruz Minas Gerais',
'Instituto Carlos Chagas':'Fiocruz Paraná',
'Gerência Regional de Brasília':'Fiocruz Brasília'}

df_test = pd.DataFrame(test_dict.items(),columns=['Affiliations','Institution'])
df_test

,Affiliations,Institution
0,Goncalo Moniz,Instituto Goncalo Moniz Fiocruz Bahia
1,Ageu Magalhaes,Instituto Aggeu Magalhães
2,Fnd Osw. Cruz,Fiocruz
3,Fundacao Osvaldo Crus,Fundacao Oswaldo Cruz
4,Instituto Leônidas e Maria Deane,Fiocruz Amazônia
5,Instituto René Rachou,Fiocruz Minas Gerais
6,Instituto Carlos Chagas,Fiocruz Paraná
7,Gerência Regional de Brasília,Fiocruz Brasília


In [113]:
predict = model.predict(df_test['Affiliations'])
predict

array(['fiocruz  ba  centro de pesquisa gonçalo moniz  cpqgm',
       'fiocruz  pe  instituto aggeu magalhães  cpqam',
       'fiocruz  sem unidade', 'fiocruz  sem unidade',
       'fiocruz  am  instituto leônidas e maria deane  ilmd',
       'fiocruz  mg  rené rachou  cpqrr',
       'fiocruz  pr  instituto carlos chagas icc',
       'fiocruz  brasília darcy ribeiro gerencia regional'], dtype=object)

In [114]:
predict = model.predict ( ['Filho cruz'. lower()])
predict[0]

'fiocruz  sem unidade'

In [115]:
predict = model.predict(['biblioteca de manguinhas'.lower()])
predict[0]

'biblioteca  fiocruz'